First goal is to find an endpoint for Form 4's

https://www.sec.gov/developer

https://www.sec.gov/os/accessing-edgar-data

In [1]:
import requests
import time
import datetime
import pandas as pd
import os
import dotenv
import xml.etree.ElementTree as ET

In [2]:
dotenv.load_dotenv()
run_once_override = False

HEADERS = {
    'User-Agent': 'Personal User {}'.format(os.getenv("EMAIL")),
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov'
}

def SEC_API_sleep():
    # Be a good citizen! Limit requests to less than 10 per second
    time.sleep(0.11)
    

Fetch the list of CIKs from here https://www.sec.gov/Archives/edgar/cik-lookup-data.txt

In [3]:
def get_url_to_outfilename(url, outfilename, run_once_override=run_once_override):
    if not run_once_override:
        return -1
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print("Status Code:", response.status_code)
    with open(outfilename, "w", encoding="utf-8") as outfile:
        outfile.write(response.text)
    SEC_API_sleep()
    return response

In [4]:
get_url_to_outfilename(
    "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt",
    "cik_list.txt"
)

-1

In [5]:
get_url_to_outfilename(
    "https://www.sec.gov/files/company_tickers.json",
    "company_tickers.json"
)

-1

In [6]:
def generate_interested_companies_dict():
    interested_company_names = []
    return_dict = dict()
    with open("interested_company_names.txt","r") as infile:
        for line in infile:
            interested_company_names.append(line.strip())
    with open("cik_list.txt","r") as infile:
        for line in infile:
            for name in interested_company_names:
                if name in line:
                    cik_number = int(line.split(":")[1])
                    cik_name = line.split(":")[0]
                    return_dict[cik_number] = {"name": cik_name}
    return return_dict

In [7]:
interested_companies_dict = generate_interested_companies_dict()

We can get information about a specific CIK's filings by querying this resource:

https://data.sec.gov/submissions/CIK##########.json

In [8]:
def save_interested_companies_indices(interested_companies_dict, run_once_override=run_once_override):
    if not run_once_override:
        return -1
    for cik in interested_companies_dict:
        url = "https://data.sec.gov/submissions/CIK{:010d}.json".format(cik)
        get_url_to_outfilename(
            "https://data.sec.gov/submissions/CIK{:010d}.json".format(cik),
            "{}.html".format(cik),
            run_once_override=run_once_override
        )
    return 0

The above URL is not fetching. Let's try something else.

https://www.sec.gov/os/accessing-edgar-data

According to the above, use https://www.sec.gov/Archives/edgar/full-index/ to get an index per quarter of all filings from all companies. Let's write a function to grab the latest index and parse out relevant links for our `interested_companies_dict`

We want the `master.idx` file which contains data in this format: `CIK|Company Name|Form Type|Date Filed|Filename`

The full path to this file looks like:

`https://www.sec.gov/Archives/edgar/full-index/{YYYY}/QTR{1-4}/master.idx`

Data extends from 1993 QTR1 - 2023 QTR1

In [9]:
def get_master_index_for_year_and_quarter(year, quarter, run_once_override=run_once_override):
    if not run_once_override:
        return -1
    get_url_to_outfilename(
        "https://www.sec.gov/Archives/edgar/full-index/{}/QTR{}/master.idx".format(year, quarter),
        "indices/{}Q{}_master.idx".format(year, quarter),
        run_once_override=run_once_override
    )
    return 0

In [10]:
get_master_index_for_year_and_quarter(2023, 1)

-1

Let's grab all indices since 1993 QTR1, then we can aggregate them together for one unified master index that we can parse for companies we're interested in.

First we'll grab all indices.

In [11]:
def get_all_master_indices(run_once_override=run_once_override, start_year=1993):
    if not run_once_override:
        return -1
    end_year = pd.Timestamp(datetime.datetime.now()).year
    end_quarter = pd.Timestamp(datetime.datetime.now()).quarter
    for year in range(start_year, end_year + 1):
        end_quarter_range = 4 if year != end_year else end_quarter
        for quarter in range(1, end_quarter_range + 1):
            print("Fetchin index for {}Q{}".format(year, quarter))
            get_master_index_for_year_and_quarter(year, quarter, run_once_override)
    return 0

In [12]:
get_all_master_indices()

-1

Let's also write a function to fetch only the latest index (for updates while the current quarter is ongoing).

In [13]:
def get_latest_master_index(run_once_override=run_once_override):
    if not run_once_override:
        return -1
    year = pd.Timestamp(datetime.datetime.now()).year
    quarter = pd.Timestamp(datetime.datetime.now()).quarter
    print("Fetchin index for {}Q{}".format(year, quarter))
    get_master_index_for_year_and_quarter(year, quarter, run_once_override)
    return 0

In [14]:
get_latest_master_index()

-1

Nice, now to read data. Here's what I want to do:

1. Read all index files.
2. Grab rows for CIKs that I'm interested in, based on `interested_companies_dict`
3. Output rows as a pandas dataframe

In [15]:
index_files = ["indices/" + filepath for filepath in os.listdir("indices")]
index_files[-1]

'indices/2023Q1_master.idx'

In [16]:
index_column_names = ["CIK","Company Name","Form Type","Date Filed","Filename"]
index_dataframe = pd.read_csv(index_files[-1], sep = "|", skiprows=11, names=index_column_names)
index_dataframe[index_dataframe["CIK"].isin(interested_companies_dict)]

,CIK,Company Name,Form Type,Date Filed,Filename
23906,1108524,"Salesforce, Inc.",10-K,2023-03-08,edgar/data/1108524/0001108524-23-000011.txt
23907,1108524,"Salesforce, Inc.",13F-HR,2023-02-10,edgar/data/1108524/0001108524-23-000005.txt
23908,1108524,"Salesforce, Inc.",3,2023-03-03,edgar/data/1108524/0001127602-23-008948.txt
23909,1108524,"Salesforce, Inc.",3,2023-03-03,edgar/data/1108524/0001127602-23-008950.txt
23910,1108524,"Salesforce, Inc.",3,2023-03-03,edgar/data/1108524/0001418812-23-000017.txt
...,...,...,...,...,...
23995,1108524,"Salesforce, Inc.",8-K,2023-01-27,edgar/data/1108524/0001193125-23-017661.txt
23996,1108524,"Salesforce, Inc.",8-K,2023-03-01,edgar/data/1108524/0001108524-23-000007.txt
23997,1108524,"Salesforce, Inc.",SC 13G/A,2023-02-07,edgar/data/1108524/0001306550-23-008329.txt
23998,1108524,"Salesforce, Inc.",SC 13G/A,2023-02-09,edgar/data/1108524/0000315066-23-002031.txt


Okay, I have the pieces to read the data correctly from the files and to focus on only the companies I care about. The last step is figure out how to concatenate two dataframes together.

Looks like `pd.concat([list of dataframes])` will do the job. Time to stitch it all together.

In [17]:
def print_percent_complete(step, total):
    print("{:.2f}% complete".format( (step / total) * 100), end = "\r")

In [18]:
def get_index_dataframe_for_interested_companies(interested_companies_dict, interested_forms=None):
    index_files = ["indices/" + filepath for filepath in os.listdir("indices")]
    index_column_names = ["CIK","Company Name","Form Type","Date Filed","Filename"]
    return_dataframe = pd.DataFrame()
    for i in range(len(index_files)):
        index_file = index_files[i]
        index_dataframe = pd.read_csv(index_file, sep = "|", skiprows=11, names=index_column_names)
        index_dataframe = index_dataframe[
            index_dataframe["CIK"].isin(interested_companies_dict)
        ]
        if interested_forms is not None:
            index_dataframe = index_dataframe[
                index_dataframe["Form Type"].isin(interested_forms)
            ]
        return_dataframe = pd.concat([return_dataframe, index_dataframe])
        print_percent_complete(i, len(index_files) - 1)
    print()
    return return_dataframe.reset_index(drop=True)

In [19]:
interested_company_form_4_index = get_index_dataframe_for_interested_companies(interested_companies_dict, ["4"])

100.00% complete


Great! I can grab the locations of the forms I want. Now to pull the data for those actual forms...

In [20]:
edgar_archive_url_root = "https://www.sec.gov/Archives/"

In [21]:
form_4_urls_to_collect = [edgar_archive_url_root + i for i in interested_company_form_4_index.tail()["Filename"]]
form_4_urls_to_collect

['https://www.sec.gov/Archives/edgar/data/1108524/0001127602-23-010343.txt',
 'https://www.sec.gov/Archives/edgar/data/1108524/0001127602-23-010346.txt',
 'https://www.sec.gov/Archives/edgar/data/1108524/0001127602-23-010481.txt',
 'https://www.sec.gov/Archives/edgar/data/1108524/0001127602-23-010750.txt',
 'https://www.sec.gov/Archives/edgar/data/1108524/0001127602-23-010933.txt']

In [26]:
def get_forms_from_url_list(form_urls_to_collect, run_once_override=run_once_override):
    if not run_once_override:
        return -1
    for i in range(len(form_urls_to_collect)):
        form_url = form_urls_to_collect[i]
        get_url_to_outfilename(
            form_url,
            "forms/" + form_url.split("/")[-1],
            run_once_override
        )
        print_percent_complete(i, len(form_urls_to_collect) - 1)
    print()
    return 0

In [27]:
get_forms_from_url_list(form_4_urls_to_collect)
form_4_paths = ["forms/" + filepath for filepath in os.listdir("forms")]
form_4_paths

['forms/0001127602-23-010343.txt',
 'forms/0001127602-23-010346.txt',
 'forms/0001127602-23-010481.txt',
 'forms/0001127602-23-010750.txt',
 'forms/0001127602-23-010933.txt']

Time to parse the XML and extract key form 4 data...

In [28]:
def get_XML_tree_from_file_path(file_path):
    file_lines = []
    with open(file_path, "r") as infile:
        file_lines = infile.readlines()
    XML_string = "".join(file_lines[file_lines.index('<XML>\n')+1:file_lines.index('</XML>\n')])
    return ET.fromstring(XML_string)

Okay we've sort of figured out how to walk through the XML tree, let's define the data we're trying to extract:

1. Name of Reporting Person
2. Title of Reporting Person
3. Issuer Name
4. Issuer Ticker
5. Non-Derivative Securities
    * Title of Security
    * Date
    * Code
    * Amount
    * Security Acquired or Disposed A / D
    * Price
    * Amount Total Following Reported Transaction
    * Ownership Direct or Indirect D / I
    * Nature of Indirect Beneficial Ownership
6. Derivative Securities
    * Title of Security
    * Exercise Price
    * Date
    * Code
    * Number Acquired
    * Number Disposed
    * Date Exercisable
    * Date of Expiry
    * Title of Security Underlying
    * Amount of Security Underlying
    * Price of Derivative Security
    * Amount of Derivative Security Total Following Reported Transaction
    * Ownership Form Direct or Indirect D / A
    * Nature of Indirect Beneficial Ownership

In [29]:
XML_tree = get_XML_tree_from_file_path(form_4_paths[-1])
[i for i in XML_tree]

[<Element 'schemaVersion' at 0x0000024AFB722E30>,
 <Element 'documentType' at 0x0000024AFB722D90>,
 <Element 'periodOfReport' at 0x0000024AFB722D40>,
 <Element 'issuer' at 0x0000024AFB722CF0>,
 <Element 'reportingOwner' at 0x0000024AFB722B10>,
 <Element 'aff10b5One' at 0x0000024AFB7223E0>,
 <Element 'nonDerivativeTable' at 0x0000024AFB722390>,
 <Element 'derivativeTable' at 0x0000024AFB746480>,
 <Element 'footnotes' at 0x0000024AFB747BF0>,
 <Element 'ownerSignature' at 0x0000024AFB747F10>]

In [57]:
def get_text_from_XML_tree_and_xpath_string(XML_tree, xpath):
    result = XML_tree.find(xpath)
    return result.text if result is not None else result

In [72]:
form_4_dict = {
    "name_of_reporting_person": 
    get_text_from_XML_tree_and_xpath_string(XML_tree, './reportingOwner/reportingOwnerId/rptOwnerName'),
    "title_of_reporting_person": 
    get_text_from_XML_tree_and_xpath_string(XML_tree, './reportingOwner/reportingOwnerRelationship/officerTitle'),
    "issuer_name":
    get_text_from_XML_tree_and_xpath_string(XML_tree, './issuer/issuerName'),
    "issuer_ticker":
    get_text_from_XML_tree_and_xpath_string(XML_tree, './issuer/issuerTradingSymbol'),
    "non_derivative_table": [
        {
            "title_of_security":
            get_text_from_XML_tree_and_xpath_string(non_derivative_transaction, './securityTitle/value'),
            "transaction_date":
            get_text_from_XML_tree_and_xpath_string(non_derivative_transaction, './transactionDate/value'),
            "transaction_code":
            get_text_from_XML_tree_and_xpath_string(non_derivative_transaction, './transactionCoding/transactionCode'),
            "amount_of_shares":
            get_text_from_XML_tree_and_xpath_string(non_derivative_transaction, './transactionAmounts/transactionShares/value'),
            "security_acquired_or_disposed":
            get_text_from_XML_tree_and_xpath_string(non_derivative_transaction, './transactionAmounts/transactionAcquiredDisposedCode/value'),
            "price":
            get_text_from_XML_tree_and_xpath_string(non_derivative_transaction, './transactionAmounts/transactionPricePerShare/value'),
        } for non_derivative_transaction in XML_tree.find('./nonDerivativeTable').findall('nonDerivativeTransaction')
    ]
}
get_text_from_XML_tree_and_xpath_string(XML_tree, '')
form_4_dict

{'name_of_reporting_person': 'Benioff Marc',
 'title_of_reporting_person': 'Chair and CEO',
 'issuer_name': 'Salesforce, Inc.',
 'issuer_ticker': 'CRM',
 'non_derivative_table': [{'title_of_security': 'Common Stock',
   'transaction_date': '2023-03-17',
   'transaction_code': 'M',
   'amount_of_shares': '725',
   'security_acquired_or_disposed': 'A',
   'price': '75.57'},
  {'title_of_security': 'Common Stock',
   'transaction_date': '2023-03-17',
   'transaction_code': 'S',
   'amount_of_shares': '341',
   'security_acquired_or_disposed': 'D',
   'price': '184.8934'},
  {'title_of_security': 'Common Stock',
   'transaction_date': '2023-03-17',
   'transaction_code': 'S',
   'amount_of_shares': '326',
   'security_acquired_or_disposed': 'D',
   'price': '185.8831'},
  {'title_of_security': 'Common Stock',
   'transaction_date': '2023-03-17',
   'transaction_code': 'S',
   'amount_of_shares': '58',
   'security_acquired_or_disposed': 'D',
   'price': '186.7848'}]}

In [66]:
XML_tree.find('./nonDerivativeTable').findall('nonDerivativeTransaction')

[<Element 'nonDerivativeTransaction' at 0x0000024AFB7222A0>,
 <Element 'nonDerivativeTransaction' at 0x0000024AFB720D60>,
 <Element 'nonDerivativeTransaction' at 0x0000024AFB7440E0>,
 <Element 'nonDerivativeTransaction' at 0x0000024AFB7454E0>]